In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from dataclasses import dataclass
from matplotlib import rc
from matplotlib import colors
import matplotlib
# #matplotlib.use('pgf')
# plt.style.use('seaborn')
# tex_fonts = {
#     # Use LaTeX to write all text
#     "text.usetex": True,
#     "font.family": "serif",
#     # Use 10pt font in plots, to match 10pt font in document
#     "axes.labelsize": 10,
#     "font.size": 10,
#     # Make the legend/label fonts a little smaller
#     "legend.fontsize": 8,
#     "xtick.labelsize": 8,
#     "ytick.labelsize": 8
# }
# plt.rcParams.update(tex_fonts)
# # plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
# # params = {'text.usetex' : True,
# #             'font.size' : 11,
# #             'font.family' : 'lmodern'}
# # plt.rcParams.update(params)
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['savefig.dpi'] = 300
# #sns.set(rc={'text.usetex' : True})

In [7]:
@dataclass
class Cell:
    def __init__(self, max_age: int):
        self.max_age = max_age
        temp_age = np.random.normal(0.5 * self.max_age, 0.2)
        self.curr_age = 1 if temp_age < 0 else temp_age
        self.carbon_mass = 0
        self.cs_rate = 0
    
    def set_cs_rate(self, density: float):
        pass
    
    def set_mass(self):
        pass
    
    def get_cs(self):
        return self.cs_rate
        
    def grow(self):
        self.curr_age += 1
        # if self.curr_age > self.max_age:
        #     self.cut()
        if self.carbon_mass + self.cs_rate < 3:
            self.carbon_mass += self.cs_rate
        
    def get_mass(self) -> float:
        return self.carbon_mass
    
    def cut(self) -> None:
        self.curr_age = 1
        self.set_mass()

def csr_mat(x):
    return (6.37*x) - (14.27*x**2) + (8.31*x**3)

class TropicalCell(Cell):
    def __init__(self, max_age: int):
        self.max_age = max_age
        temp_age = np.random.normal(0.5 * self.max_age, 0.5 * self.max_age)
        self.curr_age = 1 if temp_age < 0 else temp_age
        self.flat_rate = 0
        
    def set_cs_rate(self, density: float):
        mat_dist = csr_mat( 0.5-((0.5-(self.curr_age/self.max_age)) / 1.1))
        mat = 0.4 - (0.4 - mat_dist)/3
        mu = (0.5 * np.pi * 40 * 43 / 2000)
        self.cs_rate = self.flat_rate + 0.5 *((mat * density) * np.random.normal(mu, 0.1 * mu))
    
    def set_mass(self):
        mu = (0.5 * np.pi * 40 * 43 / 2000) * (self.curr_age / self.max_age)
        self.carbon_mass = np.random.normal(mu, 0.1 * mu) 
        
class BorealCell(Cell):
    def __init__(self, max_age: int):
        self.max_age = max_age
        temp_age = np.random.normal(0.5 * self.max_age, 0.5 * self.max_age)
        self.curr_age = 1 if temp_age < 0 else temp_age
        self.flat_rate = 0.09
        
    def set_cs_rate(self, density: float):
        mat_dist = csr_mat( 0.5-((0.5-(self.curr_age/self.max_age)) / 1.1))
        mat = 0.4 - (0.4 - mat_dist)/3
        mu = (0.5 * np.pi * (1.5**2) * 80 * 34 / 2000)
        self.cs_rate = self.flat_rate + 0.5 *(mat * 0.5 * density) * np.random.normal(mu, 0.1 * mu)
    
    def set_mass(self):
        mu = (0.5 * np.pi * (1.5**2) * 80 * 34 / 2000) * (self.curr_age / self.max_age)
        self.carbon_mass = np.random.normal(mu, 0.1 * mu) 
        
class TemperateCell(Cell):
    def __init__(self, max_age: int):
        self.max_age = max_age
        temp_age = np.random.normal(0.5 * self.max_age, 0.5 * self.max_age)
        self.curr_age = 1 if temp_age < 0 else temp_age
        self.flat_rate = 0.05
        
    def set_cs_rate(self, density: float):
        mat_dist = csr_mat( 0.5-((0.5-(self.curr_age/self.max_age)) / 1.1))
        mat = 0.4 - (0.4 - mat_dist)/3
        mu = (0.5 * np.pi * (1.25**2) * 60 * 41 / 2000)
        self.cs_rate = self.flat_rate + 0.5 * (mat * 0.5 * density) * np.random.normal(mu, 0.1 * mu)
    
    def set_mass(self):
        mu = (0.5 * np.pi * (1.25**2) * 60 * 41 / 2000) * (self.curr_age / self.max_age)
        self.carbon_mass = np.random.normal(mu, 0.1 * mu)

Creating forests given inputs

In [8]:
@dataclass
class Forest:
    def __init__(self, land, size: int, max_time: int, maturity: str, plotting: bool, rho: float, m: float, management_rate: int, ftype: str):
        self.size = size
        self.maturity = maturity
        self.plotting = plotting
        self.max_time = max_time
        self.land = land
        self.ftype = ftype
        self.management_rate = management_rate
        self.rho = rho #density threshold
        self.m = m #mass threshold
#         self.atm_carbon = [] #carbon lost to atm
        self.tot_atm_carbon = []
        self.rmvd_carbon = [] #carbon removed per deforestation
        self.tot_rmvd_carbon = [] # cumulative carbon removed
        self.curr_carbon = [] #carbon currently in forest
        self.total_carbon = [] #total carbon sequestration
        
    def simulate(self) -> None:
        p = .75
        q = 1-p
        for t in range(self.max_time):
            if t == 0:
                for (ii, ij) in np.ndindex(self.land.shape):
                    self.land[ii, ij].set_mass()
            if t % 10 == 0:
                if self.plotting == True:
                    self.plot_2d_mass(time=t)
                    plt.title('time:' + str(t))
            if t % self.management_rate == 0:                  
                curr_rmv = self.deforest()
                self.rmvd_carbon.append(p * curr_rmv)
                self.tot_rmvd_carbon.append(sum(self.rmvd_carbon) + curr_rmv)
                if self.plotting == True:
                    self.plot_2d_mass(time=t)
                    plt.title('cut time:' + str(t))
            rand_order = np.random.permutation([(x, y) for x in range(self.size) for y in range(self.size)])
            for (ii, ij) in rand_order:
                assert(isinstance(self.land[ii, ij], Cell))
                density = self.land[ii, ij].get_mass() / self.get_surrounding_density(ii, ij)
                self.land[ii, ij].set_cs_rate(density)
                self.land[ii, ij].grow()
            self.curr_carbon.append(self.get_curr_carbon_mass())
        self.total_carbon = [v + p * self.tot_rmvd_carbon[math.floor(i//self.management_rate)] for i, v in enumerate(self.curr_carbon)]
        self.total_atm_carbon = [q * self.tot_rmvd_carbon[math.floor(i//self.management_rate)] for i, v in enumerate(self.curr_carbon)]
    
        net_sequestration = [x-y for x, y in zip(self.total_carbon, self.total_atm_carbon)]

        fig, axes = plt.subplots(3,1, sharex=True, figsize=(6, 6))
        x = list(range(1, self.max_time + 1))
        sns.set_style(style = "darkgrid") 
        sns.lineplot(ax = axes[0], x=x, y=self.total_carbon, color='#81A1C1')
        sns.lineplot(ax = axes[1], x=x, y=self.total_atm_carbon, color='#81A1C1')
        sns.lineplot(ax = axes[2], x=x, y=net_sequestration, color='#81A1C1')
        plt.xlabel(r'$\mathrm{Time \ (yr)}$')
        axes[0].set(ylabel=r"$\mathrm{Total\ Carbon\ Sequestered}$")
        axes[0].yaxis.get_label().set_fontsize(10)
        axes[1].set(ylabel=r"$\mathrm{Total\ Atmospheric\ Carbon}$")
        axes[1].yaxis.get_label().set_fontsize(10)
        axes[2].set(ylabel=r"$\mathrm{Net\ Carbon\ Sequestration}$")
        axes[2].yaxis.get_label().set_fontsize(10)
        fig.align_ylabels()
        plt.tight_layout(pad=2.0)
        #plt.savefig("./images/" + self.ftype + "_carbon.pgf")
        plt.show()
        
        print("Total Carbon Sequestration: " + str(self.total_carbon[-1]))
        print("Total Carbon Lost to Atmosphere: " + str(self.total_atm_carbon[-1]))
        print("Net Sequestration: " + str(self.total_carbon[-1] - self.total_atm_carbon[-1]))
        
    def deforest(self) -> int:
        curr_rmvd = 0
        rand_order = np.random.permutation([(x, y) for x in range(2,self.size - 2) for y in range(2,self.size - 2)])
        for (ii, ij) in rand_order:
            surr_mass = self.get_surrounding_density(ii, ij) * 8
            if surr_mass > self.rho: 
                for (ix, iy) in [(x, y) for x in range(ii - 1, ii + 2) for y in range(ij - 1, ij + 2)]:
                    curr_mass = self.land[ix, iy].get_mass()
                    if curr_mass > self.m:
                        rand = np.random.random()
                        if rand > 0:
                            self.land[ix, iy].cut()
                            curr_rmvd += curr_mass
        return curr_rmvd
        
    def get_surrounding_density(self, ii: int, ij: int) -> float:
        if ii == 0: 
            if ij == 0: #upper left corner
                return (self.land[ii + 1, ij].get_mass() + self.land[ii, ij + 1].get_mass() \
                       + self.land[ii + 1, ij + 1].get_mass()) / 3
            elif ij == self.size - 1: #upper right corner
                return (self.land[ii + 1, ij].get_mass() + self.land[ii, ij - 1].get_mass() \
                       + self.land[ii + 1, ij - 1].get_mass()) / 3
            else: #first row
                return (self.land[ii, ij - 1].get_mass() + self.land[ii, ij + 1].get_mass() \
                       + self.land[ii + 1, ij - 1].get_mass() + self.land[ii + 1, ij].get_mass() \
                       + self.land[ii + 1, ij + 1].get_mass()) / 5
        elif ii == self.size - 1:
            if ij == 0: #lower left corner
                return (self.land[ii - 1, ij].get_mass() + self.land[ii, ij + 1].get_mass() \
                       + self.land[ii - 1, ij - 1].get_mass()) / 3
            elif ij == self.size - 1: #lower right corner
                return (self.land[ii - 1, ij - 1].get_mass() + self.land[ii - 1, ij].get_mass() \
                       + self.land[ii, ij - 1].get_mass()) / 3
            else: #last row
                return (self.land[ii, ij - 1].get_mass() + self.land[ii, ij + 1].get_mass() \
                       + self.land[ii - 1, ij - 1].get_mass() + self.land[ii - 1, ij].get_mass() \
                       + self.land[ii - 1, ij + 1].get_mass()) / 5
        elif ij == 0: #first column
            return (self.land[ii - 1, ij].get_mass() + self.land[ii - 1, ij + 1].get_mass() \
                   + self.land[ii, ij + 1].get_mass() + self.land[ii + 1, ij].get_mass() \
                   + self.land[ii + 1, ij + 1].get_mass()) / 5
        elif ij == self.size - 1: #last column
            return (self.land[ii - 1, ij].get_mass() + self.land[ii - 1, ij - 1].get_mass() \
                   + self.land[ii, ij - 1].get_mass() + self.land[ii + 1, ij].get_mass() \
                   + self.land[ii + 1, ij - 1].get_mass()) / 5
        else: #middle cells
            return (self.land[ii - 1, ij - 1].get_mass() + self.land[ii - 1, ij].get_mass() \
                   + self.land[ii - 1, ij + 1].get_mass() + self.land[ii, ij - 1].get_mass() \
                   + self.land[ii, ij + 1].get_mass() + self.land[ii + 1, ij - 1].get_mass() \
                   + self.land[ii + 1, ij].get_mass() + self.land[ii + 1, ij + 1].get_mass()) / 8
                
    def get_curr_carbon_mass(self) -> int:
        return sum([self.land[ii, ij].get_mass() for (ii, ij) in np.ndindex(self.land.shape)])
    
    def plot_carbon(self, pl="TCS") -> None:
        self.simulate()
#       fig, axes = plt.subplots(1, 2)
        curr_state = self.populate_state()
        sns.heatmap(curr_state, cmap="YlGnBu", ax=axes[0])
        x = list(range(1, self.max_time + 1))
        sns.set_style(style = 'whitegrid') 
        if pl == "TCS":
            sns.lineplot(ax = axes[1], x=x, y=self.total_carbon, color='#81A1C1')
            axes[1].set(xlabel='Time', ylabel='Total Carbon Sequestered')
        elif pl == "rmvd":
            x = list(range(1, self.max_time + 1, 10))
            axes[1] = sns.lineplot(x=x, y=self.rmvd_carbon, color='#81A1C1')
            axes[1].set(xlabel='Time', ylabel='Carbon Removed per Deforestation')
        elif pl == "tot_rmvd":
            x = list(range(1, self.max_time + 1, 10))
            axes[1] = sns.lineplot(x=x, y=self.tot_rmvd_carbon, color='#81A1C1')
            axes[1].set(xlabel='Time', ylabel='Total Carbon Removed')
        elif pl == "curr":
            axes[1] = sns.lineplot(x=x, y=self.curr_carbon, color='#81A1C1')
            axes[1].set(xlabel='Time', ylabel='Current Carbon Content')
        plt.xlim(0)
        plt.ylim(0)
        plt.show()
        print(self.total_carbon[-1])
        #plt.savefig("./images/" + self.ftype + "/cs.jpeg")
        
    def populate_state(self):
        ret = [[0 for _ in range(self.size)] for _ in range(self.size)]
        for i in range(self.size):
            for j in range(self.size):
                ret[i][j] = self.land[i, j].get_mass()   
        return ret
        
    def plot_2d_mass(self, time=0) -> None:
        curr_state = self.populate_state()
#         fig, axes = plt.subplots(1, 2)
        ax = sns.heatmap(curr_state, cmap = 'crest', vmin = 0, vmax = 3)
        plt.show()
        # from matplotlib.colors import LinearSegmentedColormap
        # num_colors = 6
        # col = ['#81A1C1', '#D8DEE9', '#5E81AC', '#4C566A', '#3B4252', '#2E3440']
        # #cmap = LinearSegmentedColormap.from_list('', col, num_colors)
        # ax = sns.heatmap(curr_state, square=True, cbar=False, \#cmap=camp, vmin=0, vmax=num_colors, 
        #                  xticklabels=False, yticklabels=False)
        # plt.show()
        
class Tropical(Forest):
    def __init__(self, size: int,max_time: int, maturity: str, plotting: bool, rho: float, m: float, management_rate: int, ftype="tropical"):
        super().__init__(np.array([[TropicalCell(np.random.normal(95, 15)) for _ in range(size)] for _ in range(size)]), \
                         size, max_time, maturity, plotting, rho, m, management_rate, ftype="tropical")
class Boreal(Forest):
    def __init__(self, size: int,max_time: int, maturity: str, plotting: bool, rho: float, m: float, management_rate: int, ftype="boreal"):
        super().__init__(np.array([[BorealCell(np.random.normal(150, 25)) for _ in range(size)] for _ in range(size)]), \
                         size, max_time, maturity, plotting, rho, m, management_rate, ftype="boreal")
        
class Temperate(Forest):
    def __init__(self, size: int,max_time: int, maturity: str, plotting: bool, rho: float, m: float, management_rate: int, ftype="temperate"):
        super().__init__(np.array([[BorealCell(np.random.normal(120, 20)) for _ in range(size)] for _ in range(size)]), \
                         size, max_time, maturity, plotting, rho, m, management_rate, ftype="temperate")

In [9]:
tropical = Tropical(100, 130, "test", False, 20, 2.3, 10, ftype="Tropical")
tropical.simulate()

RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1800x1800 with 3 Axes>

In [ ]:
boreal = Boreal(100, 130, "test", False, 20, 2.3, 10, ftype="Boreal")
boreal.simulate()

In [ ]:
temperate = Temperate(100, 130, "test", False, 20, 2.3, 10, ftype="Temperate")
temperate.simulate()